# Distribuições Amostrais com Dados do NHANES


Este código tem como objetivo demonstrar, de forma prática, os conceitos fundamentais de distribuições amostrais aplicados a dados reais do conjunto NHANES (National Health and Nutrition Examination Survey). A proposta é fazer a ponte entre a teoria estatística e a experimentação empírica por meio da simulação e análise computacional, mostrando como as propriedades das amostras se relacionam com a população da qual foram retiradas.


No contexto da estatística inferencial, toda análise que busca fazer inferências sobre uma população a partir de uma amostra depende de compreender o comportamento das estimativas amostrais, como médias, proporções ou desvios-padrão. Este notebook mostra, portanto, como essas estimativas variam de uma amostra para outra e como, à medida que o tamanho da amostra aumenta, suas distribuições se aproximam da distribuição normal, em conformidade com o Teorema Central do Limite.


Ao utilizar o dataset NHANES, que contém dados de saúde e características demográficas de milhares de indivíduos, o código fornece um cenário realista para simular diferentes tamanhos de amostras, calcular estatísticas descritivas e observar empiricamente a variabilidade amostral. A partir disso, o estudante visualiza na prática conceitos como:


A diferença entre população e amostra;


O papel da média amostral como estimador da média populacional;


O impacto do tamanho da amostra na precisão das estimativas;


A formação da distribuição amostral da média e sua convergência à normalidade.


Dessa forma, o notebook não apenas reforça o raciocínio teórico da inferência estatística, mas também introduz a importância da reprodutibilidade computacional no ensino de estatística. O uso de Python e bibliotecas como pandas, numpy e matplotlib permite visualizar as diferenças entre amostras, explorar as leis probabilísticas que regem o comportamento das médias e compreender como a incerteza é quantificada nas análises estatísticas.

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats.distributions import norm

In [0]:
df = spark.read.format("csv").option(
    "header", True
).load(
    "/Volumes/data_catalog_01_d/bronze/vol_data/Nhanes_2015_2016.csv"
)
display(df)

In [0]:
# Carregando dados NHANES (arquivo necessário localmente)
da = df.toPandas()
da

## Distribuição amostral da média - Subamostras de tamanho 100

In [0]:
m = 100 # tamanho da subamostra
sbp_diff = [] # lista para armazenar diferenças de médias

for i in range(1000): # número de iterações
    dx = da.sample(2 * m) # amostra aleatória de tamanho 2m
    dx1 = dx.iloc[0:m, :] # primeira subamostra
    dx2 = dx.iloc[m:, :] # segunda subamostra
    sbp_diff.append(dx1["BPXSY1"].mean() - dx2["BPXSY1"].mean()) # diferença entre médias

sns.displot(sbp_diff) # plot da distribuição das diferenças
plt.title("Diferença entre médias (subamostras de 100)") # título do gráfico
plt.show()

In [0]:
pd.Series(da["BPXSY1"]).describe() # estatísticas descritivas da variável BPXSY1

In [0]:
pd.Series(sbp_diff).describe() #checando as estatísticas do dado

## Distribuição amostral da média - Subamostras de tamanho 400

In [0]:
m = 400 # novo tamanho da subamostra
sbp_diff = [] # reiniciando a lista para diferenças de médias

for i in range(1000): # número de iterações
    dx = da.sample(2 * m) # amostra aleatória de tamanho 2m
    dx1 = dx.iloc[0:m, :] # primeira subamostra
    dx2 = dx.iloc[m:, :] # segunda subamostra
    sbp_diff.append(dx1.BPXSY1.mean() - dx2.BPXSY1.mean()) # diferença entre médias

sns.displot(sbp_diff) # plot da distribuição das diferenças
plt.title("Diferença entre médias (subamostras de 400)") # título do gráfico
plt.show()

pd.Series(sbp_diff).describe() #checando as estatísticas do dado

## Distribuição amostral do coeficiente de correlação

In [0]:
for m in [100, 400]: # diferentes tamanhos de subamostra

    sbp_diff = [] # lista para armazenar diferenças de correlações

    for i in range(1000): # número de iterações
        dx = da.sample(2 * m) # amostra aleatória de tamanho 2m
        dx1 = dx.iloc[0:m, :] # primeira subamostra
        dx2 = dx.iloc[m:, :] # segunda subamostra 
        r1 = dx1.loc[:, ["BPXSY1", "BPXDI1"]].corr().iloc[0,1] # correlação na primeira subamostra
        r2 = dx2.loc[:, ["BPXSY1", "BPXDI1"]].corr().iloc[0,1] # correlação na segunda subamostra
        sbp_diff.append(r1 - r2) # diferença entre correlações
    print(f"m={m}", "Desvio padrão empírico:", np.std(sbp_diff), "Aproximação teórica:", np.sqrt(2 / m)) # comparação do desvio padrão empírico com a aproximação teórica

## Forma da distribuição amostral da média (CLT)

In [0]:
sns.displot(da["BPXSY1"].dropna()) # plot da distribuição dos valores individuais
plt.title("Distribuição dos valores individuais de pressão sistólica") # título do gráfico
plt.show()

In [0]:
m = 100 # tamanho da amostra
sbp_mean = [] # lista para armazenar médias amostrais

for i in range(1000): # número de iterações
    dx = da.sample(m) # amostra aleatória de tamanho m 
    sbp_mean.append(dx["BPXSY1"].dropna().mean()) # média amostral

sns.displot(sbp_mean, stat="density") # plot da distribuição das médias amostrais

x = np.linspace(np.min(sbp_mean), np.max(sbp_mean), 100) # valores x para a curva normal
y = norm.pdf(x, np.mean(sbp_mean), np.std(sbp_mean)) # valores y para a curva normal
plt.plot(x, y, color='orange') # plot da curva normal
plt.title("Distribuição das médias amostrais (n=50) e curva normal") # título do gráfico
plt.show()